<a href="https://colab.research.google.com/github/MadalinaStefan/OCR_comparison/blob/main/EasyOcr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **I sistemi di riconoscimento ottico dei caratteri: confronto tra Keras Ocr, Tesseract e Easy Ocr**
















### Installazione e importazione librerie

La fase preliminare prevede l'installazione e l'importazione delle librerie necessarie.


In [1]:
!pip install keras_ocr
!pip install pybind11
!pip3 install fastwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 81.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 64.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fastwer: filename=fastwer-0.1.3-cp310-cp310-linux_x86_64.whl size=827754 sha256=a050004eb1ea80ba9a486ed061f55e790a88b4cc929c1066fad335de00dc07c5
  Stored in directory: /root/.cache/pip/wheels/99/b5/22/f6edc9a914bf3b54ebc55e258e2f9aa2297908d85799de228f
Successfully built fastwer


Il dataset di immagini e i file .csv conteneti il golden text sono situati e vengono importati da google Drive.

In [2]:
from google.colab import drive
import os
import keras_ocr
import matplotlib.pyplot as plt
import pandas as pd
import fastwer


drive.mount('/content/drive')
pipeline = keras_ocr.pipeline.Pipeline()

Mounted at /content/drive
Looking for /root/.keras-ocr/craft_mlt_25k.h5


Instructions for updating:
Use `tf.image.resize(...method=ResizeMethod.BILINEAR...)` instead.


Looking for /root/.keras-ocr/crnn_kurapan.h5


### Acquisizione delle immagini

Il codice seguente è stato utilizzato per associare a ciasciuna immagine (contenuta nel dataset immagini), il corrispettivo testo golden. Vengono create inizialmente tre liste vuote che verranno popolate con le immagini, le cartelle e il testo golden.
Ad ogni elemento presente nel dataset delle immagini, divise per tipologia, viene associato il corrispondente file .csv che contiene i testi golden di quella determinata categoria di immagini. L'associazione viene effetuata attraverso la corrispondenza dei nomi delle cartelle del dataset e dei nomi dei file .csv.
In *images_folder* vengono caricati di volta in volta i dataset nella qualità e nella trasformazione che si vuole visualizzare.

In [3]:
images_folder = '/content/drive/MyDrive/dataset_immagini_high_blur'  # Il dataset_immagini viene aggiornato ogni volta con la qualità e la trasformazione
csv_folder = '/content/drive/MyDrive/golden_text'

images = []
labels = []
true_texts = []

for folder_class in os.listdir(images_folder):
  path = csv_folder + '/' + folder_class + '.csv'
  texts = pd.read_csv(path).values


  for file in os.listdir(images_folder + '/' + folder_class):
    if file.lower().endswith(('.png', '.jpg', '.jpeg')):
      images.append(images_folder + '/' + folder_class + '/' + file)
      labels.append(folder_class)
      index = int(file.split('.')[0])
      true_texts.append(texts[index][0])


In [4]:
import numpy as np

Le immagini vengono lette attraverso keras_ocr e successivamente convertite in array.

In [5]:
images_path = images
images = [keras_ocr.tools.read(img) for img in images]
images = np.array(images, dtype=object)
images_path = np.array(images_path, dtype=object)
true_texts = np.array(true_texts, dtype=object)

# **Easy-OCR**


### Installazione

In [6]:
!pip3 install easyocr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 37.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 39.1 MB/s eta 0:00:00


### Il riconoscimento ottico dei caratteri

Il processo di riconoscimento ottico dei caratteri avviene attraverso l'utilizzo del modulo Easy Ocr: per ogni immagine l'oggetto reader defisce il testo riconosciuto al suo interno, i testi vengono concatenati in una stringa e le stringhe vengono raccolte nella lista *pred_texts* che restituisce tutti i testi predetti.

In [7]:
from sys import prefix
import os
import easyocr

def ocr_on_easy(images, language='en'):
    reader = easyocr.Reader([language])
    pred_texts = []

    for i in range(len(images)):

      result = reader.readtext(images[i])
      text = ''
      path = ''

      for detection in result:
        text += detection[1] + ' '

      pred_texts.append(text[0:len(text)-1])

    return pred_texts

### Metriche di valutazione


La seguente funzione calcola i punteggi CER e WER dei risultati prodotti dal programma, prendendo in cosiderazione e confrontanto la lista dei testi predetti con la lista dei testi golden (forniti in precedenza dall'utente).

In [8]:
def calculate_scores(pred_texts, true_texts):
  scores_cer = [ fastwer.score_sent(pred_texts[i].lower(), true_texts[i].lower(), char_level=True) for i in range(len(pred_texts)) ]

  scores_wer = [ fastwer.score_sent(pred_texts[i].lower(), true_texts[i].lower()) for i in range(len(pred_texts)) ]

  return scores_cer, scores_wer

I risultati CER e WER vengono visualizzati in file .csv per categoria di immagine. Mostrano la trasformazione eseguita e i risultati ottenuti.  

In [9]:
def generate_csv(data_class, trans, cer, wer):
  d = {'augmentation': trans, 'cer': cer, 'wer': wer}
  df = pd.DataFrame(data=d)

  csv_name = data_class + '.csv'
  df.to_csv(csv_name, sep=',', index=False)
  return df

Viene definita una seconda funzione per l'implementazione dei file .csv contenenti il percorso dell'immagine, il testo golden e il testo predetto. Questa funzione è interessante per poter visualizzare i caratteri che vengono riconosciuti dal programma, inoltre può risultare utile per comprendere il funzionamento delle operazioni CER  e WER e verificare che tra le coppie formate da testo golden e testo predetto ci sia corrispondenza.

In [10]:
def generate_text_csv(csv_label, files_path, true_text, pred_texts):
  d = {'filename': files_path, 'true_text': true_text, 'pred_text': pred_texts}
  df = pd.DataFrame(data=d)
  csv_name = csv_label + '.csv'
  df.to_csv(csv_name, sep=',', index=False)
  return df

### Risultati

Ogni immagine all'interno di una categoria di immagini viene analizzata per la qualità e la trasformazione del dataset che ha ricevuto nella fase iniziale. Vengono create le colonne dei file .csv e vengono invocate le due funzioni create precedentemente per la creazione dei file dei risultati.
I risultati CER e WER illustrano la media ottenuta dal riconoscimento di tutte le immagini di una determinata classe.

In [11]:
labels = np.array(labels)

for label in set(labels):
  print(label)
  class_index = np.where(labels==label)[0]
  test_images = []

  column_cer = []
  column_wer = []
  column_trans = []

  test_images = images[class_index]

  pred_texts = ocr_on_easy(test_images)

  cer, wer = calculate_scores(pred_texts, true_texts[class_index])

  column_cer.append(round(np.mean(cer), 2))
  column_wer.append(round(np.mean(wer), 2))
  column_trans.append("high blur") # Specifiche delle caratteristiche del dataset

      # Permette di visualizzare il testo predetto e il testo golden
  #generate_text_csv(label, images_path[class_index], true_texts[class_index], pred_texts)

      # Genera un file .csv per ogni label e permette di visualizzare i risultati CER e WER
  generate_csv(label, column_trans, column_cer, column_wer)

car_plates
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Completestructured_text
handwritten_texts
unstructured_text
